In [1]:
import os
os.environ["OPENAI_API_KEY"] = "SET_YOUR_OWN_API_KEY"

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system","You are expert in the field of traditional knowlegde.The question with four inputs named as a,b,c,d will be provided to you. you need to pice only one option from it. Display the option number and name only. After that by giving two lines space give explanation in 200 words. Follow the prompt as it is and must do all the work accordingly.Must not give heading Option: and Explanation"),
    ("user","{input}")
])

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parsers = StrOutputParser()

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('7_merged.pdf')
pages = loader.load_and_split()

In [7]:
pages

[Document(page_content="Hello learners. Welcome to the module on Indian languages.  \n \nIn this chapter, we'll introduce Indian Languages and Literature, the role of Sanskrit, \nand the significance of scriptures to current society.  \n \nIntroduction to Indian Languages \n \n  Indian languages are languages spoken in India that are classed as Indo-European \nparticularly the Indo-Iranian branch, Dravidian, Austroasiatic particularly Munda, \nand Sino-Tibetan Tibeto-Burman in particular. \n   \n  Assamese, Bengali or Bangla, Dogri, Gujarati, Hindi, Kashmiri, Konkani, Maithili, \nMarathi, Nepali, Oriya, Punjabi, Sanskrit, Sindhi, and Urdu are all members of the \nIndo-Aryan group of Indo-Iranian languages. \n  The Dravidian language family includes Kannada, Malayalam, Tamil, and Telugu.  \n   \n  The Tibeto-Burman branch of the Sino-Tibetan language family includes Manipuri \nor Meitei spoken in Manipur and Bodo spoken in northeastern India. The Munda \nlanguage family includes Santali

In [38]:
chain = prompt | llm | output_parsers

In [72]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(pages)

In [76]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [88]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""You are expert in the field of traditional knowlegde. 
The question with four inputs named as a,b,c,d will be provided to you. you need to pice only one option from it. 
Display the option number and name only. After that by giving two lines space give explanation in 100 words. 
Follow the prompt as it is and must do all the work accordingly.
Must not give heading Option: and Explanation

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [89]:
from langchain.chains import create_retrieval_chain

retriever = db.as_retriever()
retrival_chain = create_retrieval_chain(retriever,document_chain)

In [98]:
response = retrival_chain.invoke({"input":'''The gestures used to express, explain, or denote particular emotions with expressions are called
a. Norms
b. Symbols
c. Attitudes
d. Values
'''})

In [99]:
print(response["answer"])

Option: b. Symbols

Symbols are gestures or actions used to convey specific meanings or emotions. In the context of literature and drama, symbols are often used to express, explain, or denote particular emotions with expressions. These symbols can add depth and layers of meaning to a performance or written work, allowing the audience to interpret the emotions and themes being portrayed. In the context of Sanskrit dramas and ancient texts, the use of symbols through gestures was a common practice to convey intricate emotions and messages to the audience, enriching the overall storytelling experience.


# Image to text copy

In [1]:
!pip install google-generativeai

   ---------------------------------------- 0.0/45.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.3 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.3 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.3 kB 217.9 kB/s eta 0:00:01
   --------------------------- ------------ 30.7/45.3 kB 259.2 kB/s eta 0:00:01
   ---------------------------------------- 45.3/45.3 kB 318.3 kB/s eta 0:00:00


In [3]:
import google.generativeai as genai
import os 
os.environ['GOOGLE_API_KEY'] = "ENTER_YOUR_API_KEY"